In [1]:
import re
import os
import json
import numpy as np
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from utils.encode import GenericPLMEncoder

# Create the elastic instance
elastic = Elasticsearch(
    "http://localhost:9200",
    request_timeout=1000000
)

# Successful response!
elastic.info()

ObjectApiResponse({'name': 'ubuntu', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'G7JzeSt3Q5OoTS6zBdAaWQ', 'version': {'number': '8.4.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2bd229c8e56650b42e40992322a76e7914258f0c', 'build_date': '2022-08-26T12:11:43.232597118Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
import re
import json
from transformers import AutoModel, AutoTokenizer
from utils.encode import GenericPLMEncoder

plm = AutoModel.from_pretrained("/home/peitian_zhang/Code/Case-Retriever/src/data/model/DPR")
tokenizer = AutoTokenizer.from_pretrained("/home/peitian_zhang/Code/Case-Retriever/src/data/model/DPR")
model = GenericPLMEncoder(plm, tokenizer)

a = model.encode_single_query("""

2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。
庭审中，原告举示借条，该借条主要载明：今收到原告支付中交·锦悦一期二组团三标段工程5#楼（二十层-三十层）木工班组借款，金额5000元。被告在领款人签字处签名，时间为2016年8月26日。借条上有原告公章。被告质证称借条上确实是被告签名，但是出具借条时原告公司没有加盖公章，原告公司的公章是工程结束做决算时，原告在工资中抵扣了该笔款项后加盖的。
庭审中，本院向原、被告释明，本案不属于民间借贷或借款合同法律关系，原告与被告存在劳动关系，本案应属劳动争议。原告当庭表示不认可本院的释明，坚持本案属于民间借贷纠纷，仍坚持以民间借贷法律关系提出请求。
上述事实，有借条、重庆市江北区人民法院庭审笔录2份以及当事人的陈述等证据在卷佐证，并经庭审质证。

""".replace("\n", ""))

Some weights of BertModel were not initialized from the model checkpoint at /home/peitian_zhang/Code/Case-Retriever/src/data/model/DPR and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
emb = np.random.rand(768).astype(np.float32)

elastic.search(
    index="wenshu",
    size=2,
    from_=0,
    _source=False,
    fields=["case_name", "content", {"field": "publish_date", "format": "year_month_day"}, "court_name", "case_type"],
    query={
        # "match": {
        #     "content": "2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。",
        # },
        # "match_phrase": {
        #     "case_name": "彭绍波与陈忠远民间借贷纠纷一审民事判决书",
        # },
        "combined_fields": {
            "query": "借贷",
            "fields": ["case_name", "content"],
        },
        # "multi_match" : {
        #     "query": "打了一个借条",
        #     "type": "phrase",
        #     "fields": ["case_name", "content"]
        # },
        # "script_score": {
        #     "query": {
        #         "match": {
        #             "content": "2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。"
        #         }
        #     },
        #     "script": {
        #         # "source": "_score"
        #         "source": "doc['vector'].size() == 0 ? 0 : dotProduct(params.queryVector, 'vector') + 1",
        #         "params": {
        #             "queryVector": a
        #         }
        #     },
        # },
    },
    # knn={
    #     "field": "vector",
    #     "query_vector": (emb / np.linalg.norm(emb)).tolist(),
    #     "k": 10,
    #     "num_candidates": 10,
    #     # "boost": 1
    # },
    # aggs={
    #     "agg-court": {
    #         "terms": {
    #             "field": "court_name"
    #         }
    #     }
    # },
    # highlight={
    #     "fields": {
    #         "content": {
    #             "pre_tags" : ["<strong>"],
    #             "post_tags": ["</strong>"],
    #             "number_of_fragments": 1,
    #         }
    #     }
    # },
    # post_filter={
    #     "bool": {
    #         "filter": [
    #             {
    #                 "terms": {"court_name": ["上海市长宁区人民法院"]}
    #             }
    #         ]
    #     }
    # }
)["hits"]

{'total': {'value': 5741, 'relation': 'eq'},
 'max_score': 0.004156325,
 'hits': [{'_index': 'wenshu',
   '_id': 'p2-1_3547',
   '_score': 0.004156325,
   'fields': {'case_name': ['ＸＸＸ与潘超明、叶晓微民间借贷纠纷一审民事判决书'],
    'case_type': ['民事案件'],
    'court_name': ['永嘉县人民法院'],
    'publish_date': ['2017-09-29'],
    'content': ['浙江省永嘉县人民法院\n民 事 判 决 书\n（2017）浙0324民初2855号\n原告：ＸＸＸ，男，**********。\n被告：潘超明，男，**********。\n被告：叶晓微，女，**********。\n委托诉讼代理人：温从威，********。\n原告ＸＸＸ诉被告潘超明、叶晓微民间借贷纠纷一案，本院于2017年5月23日立案后，依法适用简易程序，于2017年7月17日公开开庭进行了审理。原告ＸＸＸ、被告叶晓微的委托诉讼代理人温从威到庭参加诉讼，被告潘超明经本院传票传唤无正当理由拒不到参加诉讼。本案现已审理终结。\n原告ＸＸＸ向本院提出诉讼请求：1.判令被告潘超明、叶晓微偿还借款本金187000元；2.案件受理费由被告承担。事实和理由：人人行科技股份有限公司（以下简称“人人行公司”）为一家互联网金融公司，自主开发运营手机软件借贷宝，借贷宝的注册用户可在自己的好友间自由达成借款协议。被告潘超明和原告均为借贷宝的实名注册用户。自2017年3月17日起被告潘超明在借贷宝平台上阅读并同意《借款协议》，发布借款要约，约定到期还本付息。原告在借贷宝平台上阅读并同意了《借款协议》，确认出借给被告10笔共计187000元，并通过第三方支付公司鹰皇金佰仕网络技术有限公司将上述款项划付至被告的支付账户，被告、原告及人人行公司之间的《借款协议》成立并生效。被告在借款到期后未足额还本付息。\n被告叶晓微辩称：1.《借款协议》是否真实，借款是否实际交付存疑。2.即使存在借款，也属赌债，原告有知道被告潘超明存在赌博情况的可能。3.即使借款真实合法，亦不

In [3]:
elastic.indices.create(
    index="wenshu",
    settings={
        'analysis': {
            'analyzer': {
                # we must set the default analyzer
                "default": {
                    "type": "smartcn"
                }
            }
        },
        "index.mapping.ignore_malformed": True
    },
    mappings={
        "properties": {
            # field name
            "doc_id": {
                "type": "keyword",
            },
            "court_name": {
                "type": "keyword",
            },
            "court_id": {
                "type": "keyword",
            },
            "court_province": {
                "type": "keyword",
            },
            "court_city": {
                "type": "keyword",
            },
            "court_region": {
                "type": "keyword",
            },
            "court_district": {
                "type": "keyword"
            },
            "pub_prosecution_org": {
                "type": "keyword"
            },
            "case_type": {
                "type": "keyword",
            },
            "cause": {
                "type": "keyword",
            },
            "trial_round": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
            },
            "appellor": {
                "type": "keyword"
            },
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                # enable hnsw
                "index": True,
                # inner product only allows unit-length vector
                "similarity": "dot_product"  
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'wenshu'})

In [ ]:
elastic.indices.get_mapping(index="wenshu")

In [2]:
# delete index
elastic.indices.delete(index="wenshu")

ObjectApiResponse({'acknowledged': True})

In [ ]:
elastic.indices.get_alias(index="*")

In [ ]:
elastic.delete_by_query(
    index="wenshu", 
    query={
        "match_all": {}
    }
)

In [4]:
elastic.count(
    index="wenshu",
    query={
        "match_all": {
        }
    }
)

ObjectApiResponse({'count': 990, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})